# Basic Usage of QuantLib analytics library
More details at:
https://quantlib-python-docs.readthedocs.io/en/latest/

In [24]:
import QuantLib as ql
import numpy as np
import pandas as pd

## 1. Objects and Handles
### Define a quote object and inspect the value.

In [25]:
quote = ql.SimpleQuote(.01)
quote.value()

0.01

### Use setValue() to reset values. Values will be propagated downstream in the dependency tree (using handles/smart pointers).

In [26]:
quote.setValue(.02)
quote.value()

0.02

### Define quoteHandle as a handle/smart pointer to the quote object.

In [27]:
quoteHandle = ql.QuoteHandle(quote)
quoteHandle.value()

0.02

### When the quote object is changed, the quoteHandle changes value as well.

In [28]:
quote.setValue(0.05)
quoteHandle.value()

0.05

## 2. Cashflow Schedules
### Construct a semi-annual fixed rate cashflow schedule object.

In [29]:
issue_date = ql.Date(17, 12, 2022)
maturity_date = ql.Date(17, 12, 2027)
coupon_freq = ql.Semiannual
coupon_term = ql.Period(coupon_freq)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count_conv = ql.Unadjusted
date_generation = ql.DateGeneration.Backward
month_end = False
schedule = ql.Schedule(issue_date,
                       maturity_date,
                       coupon_term,
                       calendar,
                       day_count_conv,
                       day_count_conv,
                       date_generation,
                       month_end)

### Inspect the cashflow schedule
- Use list() to get a list of all the dates in Schedule, and len() to get number of dates
- Use [] for random access
- use startDate(), endDate()

In [30]:
print("All dates: ", list(schedule))
print("Length: ", len(schedule))
print("The 3rd coupon date: ", schedule[3])  # random access
print("Start Date: ", schedule.startDate())
print("End Date: ", schedule.endDate())

All dates:  [Date(17,12,2022), Date(17,6,2023), Date(17,12,2023), Date(17,6,2024), Date(17,12,2024), Date(17,6,2025), Date(17,12,2025), Date(17,6,2026), Date(17,12,2026), Date(17,6,2027), Date(17,12,2027)]
Length:  11
The 3rd coupon date:  June 17th, 2024
Start Date:  December 17th, 2022
End Date:  December 17th, 2027


## 3. Discount Curve / Yield Curve Term Structure
### Constructing a Flat Yield Curve

In [31]:
calc_date = ql.Date(14, 4, 2023)
ql.Settings.instance().evaluationDate = calc_date

# using 5% flat interest rate for testing
flat_rate = ql.SimpleQuote(0.05)
rate_handle = ql.QuoteHandle(flat_rate)
day_count = ql.Actual360()
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
continuous_comp = ql.Continuous # continously compounded rate of 5%
flat_yield_curve = ql.FlatForward(calc_date, rate_handle, day_count, continuous_comp)
flat_yield_curve_handle = ql.YieldTermStructureHandle(flat_yield_curve)

### Inspecting the discount curve

In [32]:
ref_date = flat_yield_curve.referenceDate()
test_date = ql.Date(31, 12, 2023)

# calc year fraction between ref_date and test_date
yearFrac = flat_yield_curve.dayCounter().yearFraction(ref_date, test_date)

print("Reference Date =", ref_date)
print("Test Date =", test_date)
print("Year Fraction between Reference Date and Test Date : ", yearFrac)
print("Discount Factor for Test Date", test_date, ": ", flat_yield_curve.discount(test_date))
print("Manual DF calculation for Test Date", test_date, ": ", np.exp(-flat_rate.value() * yearFrac))
print("Difference in Discount Factor:", flat_yield_curve.discount(test_date) - np.exp(-flat_rate.value() * yearFrac))



Reference Date = April 14th, 2023
Test Date = December 31st, 2023
Year Fraction between Reference Date and Test Date :  0.725
Discount Factor for Test Date December 31st, 2023 :  0.9643991635522495
Manual DF calculation for Test Date December 31st, 2023 :  0.9643991635522495
Difference in Discount Factor: 0.0


## 4. Fixed Rate Bonds
### Constructing a bond object

In [33]:
coupon_rate = 0.04
day_count = ql.Thirty360(ql.Thirty360.USA)
coupons = [coupon_rate]

# Construct the FixedRateBond
settlement_days = 1
face_value = 100
fixed_rate_bond = ql.FixedRateBond(
    settlement_days,
    face_value,
    schedule,
    coupons,
    day_count)

### Investigate Bond Cashflows

In [34]:
x = [(cf.date(), cf.amount()) for cf in fixed_rate_bond.cashflows()]
cf_date, cf_amount = zip(*x)
cf_frame = pd.DataFrame(data={'CashFlowDate': cf_date, 'CashFlowAmount': cf_amount})
print(cf_frame)

           CashFlowDate  CashFlowAmount
0       June 20th, 2023             2.0
1   December 18th, 2023             2.0
2       June 17th, 2024             2.0
3   December 17th, 2024             2.0
4       June 17th, 2025             2.0
5   December 17th, 2025             2.0
6       June 17th, 2026             2.0
7   December 17th, 2026             2.0
8       June 17th, 2027             2.0
9   December 17th, 2027             2.0
10  December 17th, 2027           100.0


## 5. Present Value Calculation (no credit risk)
### Direct function call using risk-free bond pricing engine

In [35]:
bond_engine = ql.DiscountingBondEngine(flat_yield_curve_handle)
fixed_rate_bond.setPricingEngine(bond_engine)
pv_engine = fixed_rate_bond.NPV()
pv_engine

96.61101874409182

### Manual Calculation to verify PV

In [36]:
calc_frame = cf_frame
discount_yearfrac = np.zeros((len(calc_frame,)))
discount_factor = np.zeros((len(calc_frame,)))

i = 0
for cf_date in calc_frame['CashFlowDate']:
    discount_yearfrac[i] = flat_yield_curve.dayCounter().yearFraction(flat_yield_curve.referenceDate(), cf_date)
    discount_factor[i] = flat_yield_curve.discount(cf_date)
    i += 1

calc_frame['YearFrac'] = discount_yearfrac
calc_frame['DiscountFactor'] = discount_factor
calc_frame['NPV'] = calc_frame['CashFlowAmount'] * calc_frame['DiscountFactor']

calc_frame

,CashFlowDate,CashFlowAmount,YearFrac,DiscountFactor,NPV
0,"June 20th, 2023",2.0,0.186111,0.990738,1.981475
1,"December 18th, 2023",2.0,0.688889,0.966142,1.932284
2,"June 17th, 2024",2.0,1.194444,0.942026,1.884052
3,"December 17th, 2024",2.0,1.702778,0.918385,1.836769
4,"June 17th, 2025",2.0,2.208333,0.895461,1.790922
5,"December 17th, 2025",2.0,2.716667,0.872988,1.745976
6,"June 17th, 2026",2.0,3.222222,0.851197,1.702395
7,"December 17th, 2026",2.0,3.730556,0.829836,1.659671
8,"June 17th, 2027",2.0,4.236111,0.809122,1.618244
9,"December 17th, 2027",2.0,4.744444,0.788816,1.577632


In [37]:
pv_manual = calc_frame['NPV'].sum()
print('NPV_Engine = ', pv_engine)
print('NPV_Manual = ', pv_manual)
print('NPV diff = ', pv_manual - pv_engine)

NPV_Engine =  96.61101874409182
NPV_Manual =  96.61101874409182
NPV diff =  0.0


### Bond Clean vs Dirty Prices

In [38]:
print('Bond Notional = ', fixed_rate_bond.notional())
print('Bond Dirty Price = ', fixed_rate_bond.dirtyPrice())
print('Bond Clean Price = ', fixed_rate_bond.cleanPrice())
print('Bond Accrued = ', fixed_rate_bond.accruedAmount())


Bond Notional =  100.0
Bond Dirty Price =  96.65128172277326
Bond Clean Price =  95.31794838943992
Bond Accrued =  1.333333333333342


## 6. Market Data Scenarios
### Parallel Shift in Interest Rates Curve (can be used to compute IR01/DV01 and convexity)

In [39]:
# start with interest_rate_bump = 0
interest_rate_bump = ql.SimpleQuote(0.0)
flat_yield_curve_bumped = ql.ZeroSpreadedTermStructure(flat_yield_curve_handle, ql.QuoteHandle(interest_rate_bump))

bond_engine = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(flat_yield_curve_bumped))
fixed_rate_bond.setPricingEngine(bond_engine)

# Original price (zero interest rate bump)
print("PV for spread = 0: ", fixed_rate_bond.NPV())

# New price for +10bps interest rate  bump
interest_rate_bump.setValue(0.0010)
print("PV for spread = +10bps: ", fixed_rate_bond.NPV())

# New price for -10bps interest rate  bump
interest_rate_bump.setValue(-0.0010)
print("PV for spread = -10bps: ", fixed_rate_bond.NPV())

PV for spread = 0:  96.61101874409182
PV for spread = +10bps:  96.19593053866531
PV for spread = -10bps:  97.028018501281


## 7. Yield to Price conversions


In [40]:
# Use original interest rate yield of 5%
# flat_rate.setValue(0.05)
print('Bond PV for', flat_rate.value()*100, 'pct yield:', fixed_rate_bond.NPV())


# Change interest rate yield to 6% and recompute bond PV
flat_rate.setValue(0.06)
print('Bond PV for', flat_rate.value()*100, 'pct yield:', fixed_rate_bond.NPV())

# Set interest rate yield back to 5%
flat_rate.setValue(0.05)

Bond PV for 5.0 pct yield: 97.028018501281
Bond PV for 6.0 pct yield: 92.94298085737401


## 8. Price to Yield conversions


In [42]:
# bond_market_price = pv_engine
# bond_market_price = fixed_rate_bond.cleanPrice()
bond_market_price = 95.00

compounding = ql.Compounded

settle_date = fixed_rate_bond.settlementDate(calc_date)
day_counter = fixed_rate_bond.dayCounter()

print('day_counter =', day_counter)
print('coupon_freq =', coupon_freq)
print('calc_date =', calc_date)
print('settle_date =', settle_date)


implied_yield = fixed_rate_bond.bondYield(bond_market_price, day_counter, compounding, coupon_freq, settle_date) * 100
print('implied_yield =', implied_yield)


day_counter = 30/360 (US) day counter
coupon_freq = 2
calc_date = April 14th, 2023
settle_date = April 17th, 2023
implied_yield = 5.219382286071778
